<a href="https://colab.research.google.com/github/Kevin091295/LeetCode/blob/main/Unstructured.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is for just simple one page file ie. resume

In [9]:
!pip install -q transformers datasets peft bitsandbytes accelerate pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 61.1 MB/s eta 0:00:00


In [10]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
    return text.strip()

resume_text = extract_text_from_pdf("/content/drive/MyDrive/Kevin_Panchal.pdf")
print(resume_text[:500])  # Preview first 500 characters


Kevin Panchal
Email Id: kevinpanchal95@gmail.com| +1 (669)-338-9772 | LinkedIn: www.linkedin.com/in/kevin-panchal.
GitHub:- https://github.com/Kevin091295
SUMMARY
Seeking a full-time position in Data Science, Machine Learning, or AI Development. Skilled in Python, R, SQL, word
embedding, and CNNs, with a focus on analyzing data and developing AI-driven solutions. Currently enhancing my
expertise in large language models (LLMs), graph databases, and vector databases
EDUCATION
Cleveland State Univ


In [11]:
import json

training_data = [
    {
        "instruction": "Extract University/Colleges details from the resume.",
        "input": resume_text,
        "output": "Kevin did masters in software engineering from Cleveland State University"
    },
    {
        "instruction": "What degree does the person hold?",
        "input": resume_text,
        "output": "Master's in Software Engineering and Bachelor's in computer engineering"
    },
    {
        "instruction": "Extract work experience from the resume.",
        "input": resume_text,
        "output": "India Post Department, Business Development Officer (2018-2022)"
    },
    {
        "instruction": "Extract technical skills from the resume.",
        "input": resume_text,
        "output": "Python, Machine Learning, LLM Fine-Tuning, Django"
    },
    {
        "instruction": "Extract the person's Email from the resume.",
        "input": resume_text,
        "output": "kevinpanchal95@gmail.com"
    }
]

# Save dataset
json_path = "/content/drive/MyDrive/resume_finetune.json"
with open(json_path, "w") as f:
    json.dump(training_data, f, indent=4)

print(f"Updated dataset saved at {json_path}")


Updated dataset saved at /content/drive/MyDrive/resume_finetune.json


In [12]:
incorrect_answers = [
    {
        "instruction": "Where did the person study?",
        "input": "Resume:\n" + resume_text,
        "wrong_output": "Harvard University",
        "correct_output": "Cleveland State University"
    }
]

with open("/content/drive/MyDrive/incorrect_answers.json", "w") as f:
    json.dump(incorrect_answers, f, indent=4)

print("Incorrect answers stored for further fine-tuning.")


Incorrect answers stored for further fine-tuning.


Execute Below Code if cuda is not available

In [2]:
# !pip uninstall -y bitsandbytes
# !pip install bitsandbytes --no-cache-dir
# !pip install --upgrade transformers accelerate

# import bitsandbytes as bnb
# print("BitsAndBytes Version:", bnb.__version__)

In [14]:
import torch
from transformers import LlamaForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType

MODEL_NAME = "meta-llama/Llama-3.2-1B"
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())
print("CUDA Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# 4-bit quantization config (Reduces GPU usage)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

# Load model efficiently
model = LlamaForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

# Apply LoRA (Low-Rank Adaptation) for fine-tuning
# peft_config = LoraConfig(
#     task_type=TaskType.CAUSAL_LM,
#     r=32,  # Increased rank for better accuracy
#     lora_alpha=64,
#     lora_dropout=0.1,  # Reduces overfitting
#     target_modules=["q_proj", "v_proj"]  # Fine-tunes only attention layers
# )

# Configure LoRA for efficient fine-tuning
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,  # Low-rank adaptation
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"]  # Fine-tuning attention layers
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


CUDA Available: True
CUDA Device Count: 1
CUDA Device Name: Tesla T4
trainable params: 1,703,936 || all params: 1,237,518,336 || trainable%: 0.1377


In [15]:

from datasets import load_dataset

# Load dataset
dataset = load_dataset("json", data_files=json_path)

# Tokenization function
def tokenize_function(examples):
    inputs = tokenizer(
        examples["input"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )
    inputs["labels"] = inputs["input_ids"]
    return inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [16]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/llama_finetuned",
    per_device_train_batch_size=4,  # More examples per batch
    gradient_accumulation_steps=8,
    num_train_epochs=10,  # More epochs for better learning
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    fp16=True if torch.cuda.is_available() else False
)


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    tokenizer=tokenizer,
    data_collator=data_collator
)


<ipython-input-16-2c99f2f6f700>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [36]:
trainer.train()


Step,Training Loss


TrainOutput(global_step=5, training_loss=2.4909746170043947, metrics={'train_runtime': 8.2115, 'train_samples_per_second': 2.436, 'train_steps_per_second': 0.609, 'total_flos': 59894786949120.0, 'train_loss': 2.4909746170043947, 'epoch': 5.0})

In [17]:
trainer.save_model("/content/drive/MyDrive/llama_finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/llama_finetuned")


('/content/drive/MyDrive/llama_finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/llama_finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/llama_finetuned/tokenizer.json')

In [38]:
def ask_resume_question(question):
    input_text = f"""
### Task: Answer ONLY from the given resume.
### Resume Content:
{resume_text}

### Question:
{question}

### Answer:"""

    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=100,  # Limits output to prevent hallucination
            temperature=0.1,  # Reduces randomness for factual accuracy
            top_p=0.8,  # Limits token probability distribution
            repetition_penalty=2.1  # Prevents repeating irrelevant details
        )

    return tokenizer.decode(output[0], skip_special_tokens=True).strip()

# Example Queries
print(ask_resume_question(" Did kevin masters in GTU?"))












Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


### Task: Answer ONLY from the given resume.
### Resume Content:
Kevin Panchal
Email Id: kevinpanchal95@gmail.com| +1 (669)-338-9772 | LinkedIn: www.linkedin.com/in/kevin-panchal.
GitHub:- https://github.com/Kevin091295
SUMMARY
Seeking a full-time position in Data Science, Machine Learning, or AI Development. Skilled in Python, R, SQL, word
embedding, and CNNs, with a focus on analyzing data and developing AI-driven solutions. Currently enhancing my
expertise in large language models (LLMs), graph databases, and vector databases
EDUCATION
Cleveland State University Jan 2023-Dec 2024
Master of Software Engineering
Relevant Coursework: Deep Learning, Software Engineering, Database Management, Data mining
Vishwakarma Government Engineering College, Gujarat Technological University Aug 2013-May 2017
Bachelors in Computer Engineering
Relevant Coursework: Computer Network, System Programming, Artificial Intelligence, Data Structure
TECHNICAL SKILLS
● Current Learning Focus: Fine-tuning Llama